<center><br><font size=12> Big data course - project 1 </font><br>
    <br>
<br>
<b> Yahlly schein <b>     <br>
<b> Liya Gurevitch<b>    <br>
<b> 7.2.2024  <b>    <br>

 # Assumptions
 
 ## General
1. If the redis was restated (for any reason) the recovery function will run automaticlly.
2. We save a partial inforamtion in the redis for the queries in the system - O(1) operation
3. All the information is stored in MongoDB
4. We have saved another attribute (Messgae ID) for "send message" function so we will be able to store it in redis

## Specific "edge cases":
1. Some edge cases tests are implemented in the code with conditions - Michael said it won't be the main thing he is going to check
2. Remove device function - a user wouldnt ask to delete a device which is not on his devices list
3. Start conversation function - a conversation will be initiated by a device which belongs to the user!
4. In the function "count conversations by device type" we are only interested in devices that are in the system (and haven't been deleted). If we hadn't took this assumption we could have easily save the initiating device OS type as another attribute of the conversation and sum all there values...

# Settings

In [ ]:
# ! pip install pymongo
# ! pip install redis

## Imports

In [10]:
import redis
import pymongo
from pymongo import MongoClient

import pandas as pd
import pprint
from datetime import datetime
import time
import json

## redis and mongodb info 

In [11]:
# mongo info:
m_username = "bdl-mongodb-group11"
m_password = "xrQkHaWBCmsFjZ1Y"
m_group = "group11"


#redis info:
r_host = 'redis-15757.c321.us-east-1-2.ec2.cloud.redislabs.com'
r_port = 15757
r_username = "bdl-redis-group11"
r_passwd = "h2Ey5S6$u$"
r_decode_responses=True

## Initialization

In [12]:
def redis_connection(host_name=r_host, passwd=r_passwd, username=r_username, port_num = r_port):
  """
  connecting to the redis with the given host and passward for group 11
  """
  r = redis.Redis(host=host_name, 
                  port=port_num,
                  username=username,
                  password=passwd,
                  decode_responses=True)
  return r

 
def mongo_connection(username=m_username, passwd=m_password):
  client = MongoClient("mongodb+srv://{}:{}@cluster0.xzksgtp.mongodb.net/?retryWrites=true&w=majority".format(username, passwd))
  return client

In [13]:
def setup_mongo(client, group):
    db = client[group]
    users_collection = db.users
    conversations_collection = db.conversations
    print("users_collection created in {} db".format(group))
    print("conversations_collection created in {} db".format(group))
    return db, users_collection, conversations_collection

def reset_redis(r):
    r.flushdb()
    print('redis database is clean')

def reset_mongo(client, group="group11"):
    client[group].users.drop()
    client[group].conversations.drop()
    print("mongo is clean")

def restart():
    # connect to dbs
    r = redis_connection()
    client = mongo_connection()
    # clean dbs
    reset_redis(r)
    reset_mongo(client)
    # setup the databases
    db, users_collection, conversations_collection = setup_mongo(client, "group11")
    print("restart completed")
    return r, client, db, users_collection, conversations_collection

## Restart everything

In [14]:
r, client, db, users_collection, conversations_collection = restart()

redis database is clean
mongo is clean
users_collection created in group11 db
conversations_collection created in group11 db
restart completed


# Operations

## add user function

In [18]:
# ID will be saved in redis and mongo will contain all the info including the ID.
def add_user(user):
    if not r.exists('group11:last_user_id'):
        r.set('group11:last_user_id', 0)
        
    new_id = r.incr('group11:last_user_id')
    new_date = datetime.now().strftime("%d/%m/%Y")
    
    user['user_id'] = new_id
    user['system_joining_date'] = new_date
    user['devices'] = [] # initiate devices list
    
    users_collection.insert_one(user)

    r.set(f'group11:user:user_id:{user["user_id"]}', user['user_id'])
    print(f"{user['user_first_name']} {user['user_last_name']} Added successfully!")
    return new_id

In [20]:
new_user_id = add_user({'user_first_name': 'John', 'user_last_name': 'Snow'})
print(f"New user added with ID: {new_user_id}")

John Snow Added successfully!
New user added with ID: 3


In [17]:
a = users_collection.find_one({"user_id": 1})
pprint.pprint(a)

{'_id': ObjectId('65ca28ca6ac98da4ac21732f'),
 'devices': [],
 'system_joining_date': '12/02/2024',
 'user_first_name': 'John',
 'user_id': 1,
 'user_last_name': 'Snow'}


## register device function

In [21]:
# ID will be saved in redis and mongo will contain all the info including the ID.
def register_device(device, user_id):
    if not r.exists('group11:last_device_id'):
        r.set('group11:last_device_id', 0)
        
    new_id = r.incr('group11:last_device_id')
    new_date = datetime.now().strftime("%d/%m/%Y")
    
    device['device_id'] = new_id
    device['device_registration_date'] = new_date
    users_collection.update_one({"user_id": user_id}, {"$push": {"devices": device}})
    
    # Set in redis (so "remove_device" function could delete it in O(1))
    r.set(f'group11:devices:user_id:{user_id}:device_id:{new_id}', new_id)
    
    print(f"{device['device_name']} {device['device_os_type']} {device['device_manufacturer_name']} Added successfully!")
    return new_id

In [45]:
new_device_id = register_device(device={'device_name': 'MySamsung', 'device_os_type': 'Android',
'device_manufacturer_name': 'Samsung'}, user_id=1)
print(f"New device added with ID: {new_device_id}")

MySamsung Android Samsung Added successfully!
New device added with ID: 5


In [46]:
a = users_collection.find_one({"user_id": 1})
pprint.pprint(a)

{'_id': ObjectId('65ca28ca6ac98da4ac21732f'),
 'devices': [{'device_id': 1,
              'device_manufacturer_name': 'Samsung',
              'device_name': 'MySamsung',
              'device_os_type': 'Android',
              'device_registration_date': '12/02/2024'},
             {'device_id': 2,
              'device_manufacturer_name': 'Samsung',
              'device_name': 'MySamsung',
              'device_os_type': 'Android2',
              'device_registration_date': '12/02/2024'},
             {'device_id': 5,
              'device_manufacturer_name': 'Samsung',
              'device_name': 'MySamsung',
              'device_os_type': 'Android',
              'device_registration_date': '12/02/2024'}],
 'system_joining_date': '12/02/2024',
 'user_first_name': 'John',
 'user_id': 1,
 'user_last_name': 'Snow'}


## start conversation function

In [26]:
# ID and status will be saved in redis and mongo will contain all the info.
def start_conversation(user_group, initiating_user_id, initiating_user_device_id):
    # a user group can contain at most 10 users.
    if len(user_group["user_ids"]) > 10:
        raise ValueError(f"The user group {user_group['group_name']} can contain 10 users at most - remove some")
    if initiating_user_id not in user_group["user_ids"]:
        raise ValueError(f"The user {initiating_user_id} is not in the group he wants to create - forbidden")
    if not r.exists('group11:last_conversation_id'):
        r.set('group11:last_conversation_id', 0)
        
    new_id = r.incr('group11:last_conversation_id')
    new_date = datetime.now().strftime("%d/%m/%Y %H:%M")
    
    conversation = {}
    conversation['conversation_id'] = new_id
    conversation['conversation_start_datetime'] = new_date
    conversation['initiating_user_id'] = initiating_user_id
    conversation['initiating_user_device_id'] = initiating_user_device_id
    conversation['user_group'] = user_group
    conversation['messages'] = [] # initiate messages list
    
    conversations_collection.insert_one(conversation)
    
    # Set the values of the converation in redis (id and ststus)
    r.set(f'group11:conversation:conversation_id:{new_id}', new_id)
    r.set(f'group11:conversation:conversation_status:{new_id}', "open")
    
    
    # Insert into redis the users in the conversation
    for user_id in user_group["user_ids"]:
        r.set(f'group11:user_in_conv:conversation_id:{new_id}:user_id:{user_id}', user_id)
        
        
    # set/ update a counter for report 4.1 (num of conversations)
    for user_id in user_group["user_ids"]:
        active_conversations_key = f'group11:user:{user_id}:active_conversations'
        if not r.exists(active_conversations_key):
            r.set(active_conversations_key, 1)
        else:
            r.incr(active_conversations_key)
            
            
    # set a counter for report 4.2 (max num of messgaes)
    max_messages_key = 'group11:max_num_messages'
    if not r.exists(max_messages_key):
        r.set(max_messages_key, 0)
  
    print(f"{user_group} Added successfully!")
    return new_id

In [29]:
new_conversation_id = start_conversation(user_group={'group_name': 'Friends3', 'user_ids': [1, 2, 3]},
initiating_user_id=1, initiating_user_device_id=1)

print(f"New conversation added with ID: {new_conversation_id}")

{'group_name': 'Friends3', 'user_ids': [1, 2, 3]} Added successfully!
New conversation added with ID: 3


In [24]:
a = conversations_collection.find_one({"conversation_id": 1})
pprint.pprint(a)

{'_id': ObjectId('65c8a446b649b1988dc15360'),
 'conversation_id': 1,
 'conversation_start_datetime': '11/02/2024 12:41',
 'initiating_user_device_id': 1,
 'initiating_user_id': 1,
 'messages': [],
 'user_group': {'group_name': 'Friends', 'user_ids': [1, 2, 3]}}


## end conversation function

In [30]:
# status will be updated in both redis and details in mongo.
def end_conversation(conversation_id, closing_user_id):
    if r.get(f"group11:conversation:conversation_status:{conversation_id}") == "closed":
        raise ValueError("This converation is already ended")
    if not r.exists(f'group11:conversation:conversation_id:{conversation_id}'):
        raise ValueError("You can't delete a conversation that does not exist.")
    if not r.exists(f'group11:user:user_id:{closing_user_id}'):
        raise ValueError("An unexisting user cant make any actions.")
        
    ending_date = datetime.now().strftime("%d/%m/%Y %H:%M")
    
    #set in mongo
    conversations_collection.update_one({"conversation_id": conversation_id}, {"$set": {"conversation_end_datetime": ending_date}})
    conversations_collection.update_one({"conversation_id": conversation_id}, {"$set": {"closing_user_id": closing_user_id}})
    
    #set in redis
    r.set(f'group11:conversation:conversation_status:{conversation_id}', 'closed')
    
    # update the counter for report 4.1 (num of conversations) -1
    for user_id in conversations_collection.find_one({"conversation_id": 1})['user_group']['user_ids']:
        active_conversations_key = f'group11:user:{user_id}:active_conversations'
        r.decr(active_conversations_key)
            
    print(f"Conversation {conversation_id} ended successfully!")
    return conversation_id

In [31]:
ending_conversation_id = end_conversation(conversation_id=2, closing_user_id=1)
print(f"New conversation ended with ID: {ending_conversation_id}")

Conversation 2 ended successfully!
New conversation ended with ID: 2


In [32]:
a = conversations_collection.find_one({"conversation_id": 2})
pprint.pprint(a)

{'_id': ObjectId('65ca29256ac98da4ac217333'),
 'closing_user_id': 1,
 'conversation_end_datetime': '12/02/2024 16:20',
 'conversation_id': 2,
 'conversation_start_datetime': '12/02/2024 16:20',
 'initiating_user_device_id': 2,
 'initiating_user_id': 1,
 'messages': [],
 'user_group': {'group_name': 'Friends', 'user_ids': [1, 2, 3]}}


## send message function

In [33]:
## This operation is performed at a run time complexity of O(1) in redis only (as required).
def send_message(user_id, conversation_id, message_content):
    if not r.exists(f'group11:conversation:conversation_id:{conversation_id}'):
        raise ValueError(f"The conversation {conversation_id} is not in the data base")
        
    # This operation is valid only if the conversation has not been closed    
    if r.get(f'group11:conversation:conversation_status:{conversation_id}') == "closed":
        raise ValueError(f"The conversation {conversation_id} is closed. You can't send a message in a closed group")
    
    # initiate an id for saving in redis
    if not r.exists('group11:last_message_id'):
        r.set('group11:last_message_id', 0)
    message_id = r.incr('group11:last_message_id')
    
    message_date = datetime.now().strftime("%d/%m/%Y %H:%M")
    
    message = {}
    message['user_id'] = user_id
    message['message_content'] = message_content
    message['message_datetime'] = message_date
    message['message_id'] = message_id
    
    #set in mongo
    conversations_collection.update_one({"conversation_id": conversation_id}, {"$push": {"messages": message}})
    
    #set in redis (in O(1))
    r.set(f'group11:message:message_id:{message_id}', message_content)
    
    
    # Calculations for report 4.2
    num_messages_key = f'group11:user:{user_id}:num_messages'
    if not r.exists(num_messages_key):
        r.set(num_messages_key, 1)
    else:
        r.incr(num_messages_key)
    
    current_user_num_messages = int(r.get(num_messages_key))
    if current_user_num_messages > int(r.get('group11:max_num_messages')):
        r.set('group11:max_num_messages', current_user_num_messages)
        r.set('group11:user:max_num_messages', user_id)
    
    print(f"User {user_id} sent a meaasge to conversation {conversation_id} successfully!")
    return message_id

In [38]:
messgae_id = send_message(user_id=2, conversation_id=1, message_content='Hi everyone, it is great to join the conversation!!!')
print(f"New message sent with ID: {messgae_id}")

User 2 sent a meaasge to conversation 1 successfully!
New message sent with ID: 5


In [39]:
a = conversations_collection.find_one({"conversation_id": 1})
pprint.pprint(a)

{'_id': ObjectId('65ca291b6ac98da4ac217332'),
 'conversation_id': 1,
 'conversation_start_datetime': '12/02/2024 16:20',
 'initiating_user_device_id': 1,
 'initiating_user_id': 1,
 'messages': [{'message_content': 'Hi everyone, it is great to join the '
                                  'conversation!!!',
               'message_datetime': '12/02/2024 16:21',
               'message_id': 1,
               'user_id': 1},
              {'message_content': 'Hi everyone, it is great to join the '
                                  'conversation!!!',
               'message_datetime': '12/02/2024 16:21',
               'message_id': 4,
               'user_id': 1},
              {'message_content': 'Hi everyone, it is great to join the '
                                  'conversation!!!',
               'message_datetime': '12/02/2024 16:21',
               'message_id': 5,
               'user_id': 2}],
 'user_group': {'group_name': 'Friends', 'user_ids': [1, 2, 3]}}


## remove device function

In [40]:
## This operation is performed at a run time complexity of O(1) in redis only (as required).
def remove_device(user_id, device_id):
    #Find the user and check if the device exists in the 'devices' array
    user_device = users_collection.find_one({"user_id": user_id, "devices.device_id": device_id})
    if not user_device:
        raise ValueError(f"Device {device_id} was not found for User {user_id} therefor it cant be deleted")
   
    #delete from mongo
    users_collection.update_one({"user_id": user_id}, {"$pull": {"devices": {"device_id": device_id}}})
    
    #delete from redis in O(1)
    r.delete(f'group11:devices:user_id:{user_id}:device_id:{device_id}')

    print(f"Device {device_id} deleted from User {user_id} successfully!")
    return device_id

In [47]:
device_id = remove_device(user_id=1, device_id=5)
print(f"The device {device_id} has been deleted")

Device 5 deleted from User 1 successfully!
The device 5 has been deleted


In [48]:
a = users_collection.find_one({"user_id": 1})
pprint.pprint(a)

{'_id': ObjectId('65ca28ca6ac98da4ac21732f'),
 'devices': [{'device_id': 1,
              'device_manufacturer_name': 'Samsung',
              'device_name': 'MySamsung',
              'device_os_type': 'Android',
              'device_registration_date': '12/02/2024'},
             {'device_id': 2,
              'device_manufacturer_name': 'Samsung',
              'device_name': 'MySamsung',
              'device_os_type': 'Android2',
              'device_registration_date': '12/02/2024'}],
 'system_joining_date': '12/02/2024',
 'user_first_name': 'John',
 'user_id': 1,
 'user_last_name': 'Snow'}


## remove user from conversation group function

In [49]:
## This operation is performed at a run time complexity of O(1) in redis only (as required).
def remove_user_from_conversation_group(conversation_id, user_id):
    users_in_converstaion = conversations_collection.find_one({"conversation_id": conversation_id})["user_group"]['user_ids']
    if user_id not in users_in_converstaion:
        raise ValueError(f"The user {user_id} is not in the conversation {conversation_id}")
    
    #delete from mongo
    conversations_collection.update_one({"conversation_id": conversation_id}, {"$pull": {"user_group.user_ids": user_id}})
    
    #delete from redis in O(1)
    r.delete(f'group11:user_in_conv:conversation_id:{conversation_id}:user_id:{user_id}')
    
    #decrise the number of active conversations of this user by 1 (for report 1)
    active_conversations_key = f'group11:user:{user_id}:active_conversations'
    r.decr(active_conversations_key)
    
    print(f"user {user_id} deleted from conversation {conversation_id} successfully!")
    return conversation_id

In [50]:
conversation_id = remove_user_from_conversation_group(conversation_id=1, user_id=2)
print(f"The user has been deleted")

user 2 deleted from conversation 1 successfully!
The user has been deleted


In [51]:
a = conversations_collection.find_one({"conversation_id": 1})
pprint.pprint(a)

{'_id': ObjectId('65ca291b6ac98da4ac217332'),
 'conversation_id': 1,
 'conversation_start_datetime': '12/02/2024 16:20',
 'initiating_user_device_id': 1,
 'initiating_user_id': 1,
 'messages': [{'message_content': 'Hi everyone, it is great to join the '
                                  'conversation!!!',
               'message_datetime': '12/02/2024 16:21',
               'message_id': 1,
               'user_id': 1},
              {'message_content': 'Hi everyone, it is great to join the '
                                  'conversation!!!',
               'message_datetime': '12/02/2024 16:21',
               'message_id': 4,
               'user_id': 1},
              {'message_content': 'Hi everyone, it is great to join the '
                                  'conversation!!!',
               'message_datetime': '12/02/2024 16:21',
               'message_id': 5,
               'user_id': 2}],
 'user_group': {'group_name': 'Friends', 'user_ids': [1, 3]}}


# Reports

## show number of conversations function

In [52]:
# Just return the counter... It has been implemented before
def show_number_of_conversations(user_id):
    active_conversations_key = f'group11:user:{user_id}:active_conversations'
    return r.get(active_conversations_key)

In [55]:
show_number_of_conversations(1)

'2'

## show most active user function

In [56]:
def show_most_active_user():
    return r.get('group11:user:max_num_messages')

In [57]:
show_most_active_user()

'1'

## count conversations by device type

In [58]:
def count_conversations_by_device_type():
    pipeline = [
        {"$lookup": {
            "from": "users",
            "localField": "initiating_user_id",
            "foreignField": "user_id",
            "as": "user_info"}},
        
        {"$unwind": "$user_info"},
        {"$unwind": "$user_info.devices"},
        {"$match": 
                 {"$expr": 
                    {"$eq": ["$initiating_user_device_id", "$user_info.devices.device_id"]}}},
        {"$group": {
            "_id": "$user_info.devices.device_os_type",
            "count": {"$sum": 1}
        }},
        {"$sort": {"count": -1}}
    ]
    
    result = conversations_collection.aggregate(pipeline)
    df = pd.DataFrame(list(result))
    
    return df

In [60]:
count_conversations_by_device_type()

,_id,count
0,Android2,2
1,Android,1


## top long conversations between dates

In [61]:
def top_long_conversations_between_dates(start_date, end_date):
    pipeline = [
        {"$match": {"$and": [{"conversation_start_datetime": {"$gte": start_date, "$lte": end_date}},
                              {"conversation_end_datetime": {"$exists": True}}]}},
        {"$project": {"_id": 0,
            "conversation_id": 1,
            "conversation_length": {"$subtract": [
                {"$dateFromString": {"dateString": "$conversation_end_datetime"}},
                {"$dateFromString": {"dateString": "$conversation_start_datetime"}}]}}},
        {"$sort": {"conversation_length": -1}},
        {"$limit": 5}]
    result = conversations_collection.aggregate(pipeline)

    df = pd.DataFrame(list(result))
    if df.empty:
        return "No conversations at this period of time"
    else:
        return list(df["conversation_id"])

In [62]:
list_of_conversations = top_long_conversations_between_dates('04/01/2024', '15/02/2024')
print(f"The 5 longest conversations (or less if there are not 5) are: {list_of_conversations}")

The 5 longest conversations (or less if there are not 5) are: [2]


# Recovery

## redis power failure function

In [63]:
def redis_power_failure(r):
  r.flushdb()
  print("The redis info is gone :O")

In [64]:
redis_power_failure(r)

The redis info is gone :O


## recovery

In [65]:
def recovery():
    # connect to redis back "again" and create a new redis instance
    r = redis_connection()
    print("---------------------------")
    print("Recovery from Mongo db has started")
    print("---------------------------")
    
    
    # find the larest user_id to keep the same indexing 
    pipeline = [{"$group": {"_id": None, "max_user_id": {"$max": "$user_id"}}}]
    result = list(users_collection.aggregate(pipeline))
    max_user_id = result[0]["max_user_id"] if result else 0
    ## Set in Redis
    r.set('group11:last_user_id', max_user_id)
    print(f"The last user id in the system is {max_user_id}")
    
    
    # find the larest device_id to keep the same indexing 
    pipeline = [{"$unwind": "$devices"},
                {"$group": {"_id": None, "max_device_id": {"$max": "$devices.device_id"}}}]
    result = list(users_collection.aggregate(pipeline))
    max_device_id = result[0]["max_device_id"] if result else 0
    ## Set in Redis
    r.set('group11:last_device_id', max_device_id)
    print(f"The last device id in the system is {max_device_id}")
    
    
    # find the larest conversation_id to keep the same indexing 
    pipeline = [{"$group": {"_id": None, "max_conversation_id": {"$max": "$conversation_id"}}}]
    result = list(conversations_collection.aggregate(pipeline))
    max_conversation_id = result[0]["max_conversation_id"] if result else 0
    ## Set in Redis
    r.set('group11:last_conversation_id', max_conversation_id)
    print(f"The last conversation id in the system is {max_conversation_id}")
    
    
    # Define the pipeline for 'group11:user:{user_id}:active_conversations'
    # filter where the conversation_end_datetime is null (which means it still exists)
    # count active conversations for each user
    pipeline = [
        {"$match": {"conversation_end_datetime": None}},
        {"$unwind": "$user_group.user_ids"},
        {"$group": {"_id": "$user_group.user_ids", "active_conversations": {"$sum": 1}}}]
    ## Set in Redis
    for result in conversations_collection.aggregate(pipeline):
        user_id = result["_id"]
        active_conversations = result["active_conversations"]
        redis_key = f"group11:user:{user_id}:active_conversations"
        r.set(redis_key, active_conversations)
    print("All users and their active conversations number have been added") 
    
        
    # Define the pipeline to count messages for each user
    pipeline = [
        {"$unwind": "$messages"},
        {"$group": {"_id": "$messages.user_id", "num_messages": {"$sum": 1}}},
        {"$sort": {"num_messages": -1}},
        {"$limit": 1}]
    result = conversations_collection.aggregate(pipeline)
    max_messages_user = next(result, None)
    if max_messages_user:
        user_id_with_max_messages = max_messages_user["_id"]
        max_num_messages = max_messages_user["num_messages"]
        r.set('group11:max_num_messages', max_num_messages)
        r.set('group11:user:max_num_messages', user_id_with_max_messages)
    else:
        r.set('group11:max_num_messages', 0)
    print(f"The max num of messages is {max_num_messages} by user {user_id_with_max_messages}") 
        
        
    # Iterate over all users and define the redis keys for users and user_devices 
    for user in users_collection.find():
        user_id = user['user_id']
        r.set(f'group11:user:user_id:{user_id}', user_id)
        for device in user['devices']:
            device_id = device['device_id']
            key = f"group11:devices:user_id:{user_id}:device_id:{device_id}"
            value = device_id
            r.set(key, value)
    print("All users have been added") 
    print("All users' devices have been added")  

    
    # Define the pipeline to set conversation status for each conversation in redis
    pipeline = [
        {"$project": {"conversation_id": 1, "conversation_end_datetime": 1}},
        {"$addFields": {
            "conversation_status": {
                "$cond": {
                    "if": {"$eq": [{"$type": "$conversation_end_datetime"}, "missing"]},
                    "then": "open",
                    "else": "closed"}}}}]  
    result = conversations_collection.aggregate(pipeline)
    for conversation in result:
        conversation_id = conversation["conversation_id"]
        conversation_status = conversation["conversation_status"]
        # Set the 'group11:conversation:conversation_status:{conversation_id}' in Redis
        r.set(f'group11:conversation:conversation_status:{conversation_id}', conversation_status)
        # Set the 'group11:conversation:conversation_id:{new_id}' in Redis
        r.set(f'group11:conversation:conversation_id:{conversation_id}', conversation_id) 
    print("All conversations' statuses have been added")  
        
        
    # Define the pipeline to unwind user_ids from the user group to set these values in redis
    pipeline = [{"$unwind": "$user_group.user_ids"},
                {"$project": {"conversation_id": 1, "user_id": "$user_group.user_ids"}}]
    result = conversations_collection.aggregate(pipeline)
    for doc in result:
        conversation_id = doc["conversation_id"]
        user_id = doc["user_id"]
        # Set the redis instance for the user in the conversation
        r.set(f'group11:user_in_conv:conversation_id:{conversation_id}:user_id:{user_id}', user_id)    
    print("All users ids in conversations have been added")     
    
    
    # Find the largest message_id to keep same indexing
    pipeline = [{"$unwind": "$messages"}, {"$group": {"_id": None, "max_message_id": {"$max": "$messages.message_id"}}}]
    result = list(conversations_collection.aggregate(pipeline))
    max_message_id = result[0]["max_message_id"] if result else 0
    # Set the largest message_id in Redis
    r.set('group11:last_message_id', max_message_id)
    print(f"The last message id in the system is {max_message_id}")

    
    # Define the pipeline to unwind messages from each conversation to set these values in redis
    pipeline = [{"$unwind": "$messages"},
                {"$project": {"message_id": "$messages.message_id", "message_content": "$messages.message_content"}}]
    result = conversations_collection.aggregate(pipeline)
    for doc in result:
        message_id = doc["message_id"]
        message_content = doc["message_content"]
        # Set the redis instance for the message
        r.set(f'group11:message:message_id:{message_id}', message_content)
    print("All messages have been added")
    print("---------------------------")
    print("---DONE---")  
    print("---------------------------")

    return "All good - no worries"

In [66]:
recovery()

---------------------------
Recovery from Mongo db has started
---------------------------
The last user id in the system is 3
The last device id in the system is 3
The last conversation id in the system is 3
All users and their active conversations number have been added
The max num of messages is 4 by user 1
All users have been added
All users' devices have been added
All conversations' statuses have been added
All users ids in conversations have been added
The last message id in the system is 5
All messages have been added
---------------------------
---DONE---
---------------------------


'All good - no worries'

# Modeling description

In this implementation, both Redis and Mongo db are utilized for different purposes in managing and storing data as we discussed in class.

All data is stored in the Mongo db and some of it also in the redis db (explained below).

Mongo db serves as the primary database for storing structured data, while redis complements Mongo db by providing caching and efficient data access for specific use cases.

Since we are not allowed to create more than 2 collections, we have decided on the 2 logical collections.
The collections store various types of data related to users (including user's devices) and conversations (including messages).

The users_collection stores information about users such as user ID, user name, user devices, and system joining date.

The conversations_collection stores information about conversations, including conversation ID, participating users, start and end datetime, and messages exchanged within each conversation.


Redis is used as a caching and key-value store to optimize specific operations that were requiered to be excecuted in O(1), save counters for the reports and improve performance.

Redis is used to store the last user ID, last message ID, and other metadata required for efficient data retrieval and management.
